# How to create a company house developer account:

note: depending on your setup, links might not work when you click them. You can try right-click and 'copy link address', or just copy-pasting these links.

- go into companies house website https://developer.company-information.service.gov.uk/api/docs/index/gettingStarted/quickStart.html
- click Register, enter your school email and then click the link you received by email
- setup a password
- login to  https://developer.company-information.service.gov.uk/developer/signin
- go to Your Applications on top (https://developer.company-information.service.gov.uk/developer/applications)
- choose Register Your Application > REST API key
- give your application a name, like "UoE Business School Project" > Register. Don;t write anything in the IP addresses part.
- you will see API key value. Copy it into your notebook. It will look like this: kGTMYdddAmrSDPviC0sFP48zyQ47tKxd_ikP1345

# How to use the company house API:

- you are allowed to ask 600 requests within a five-minute period, after which you get another 600. Then it will make you wait for a few seconds/minutes (https://developer.company-information.service.gov.uk/api/docs/index/gettingStarted/rateLimiting.html)

- you can use the Documentations to preview and prototype all possible API calls. For example go to https://developer.company-information.service.gov.uk/api/docs/search/companies/companysearch.html (you need to be logged in to prototype). eg. Use the prototyping tool to find all companies with a work 'Bagel' in the name
- you can also see who owns the company. eg. grab one of the company_number values from the above 'Bagel' search and use it in the people search (for example '06882845' ): https://developer.company-information.service.gov.uk/api/docs/company/company_number/persons-with-significant-control/listPersonsWithSignificantControl.html

# Example tasks to work on today's lab in your notebook

(do these in here, using python, not inside of the website prototyping tool)

- find all information of companies with a word 'Bagel' in the name (or just use any other word)
- from all those companies, how would you get those that are still active? can you calculate how old they are on average?
- and for those that are not active, how long did they exist? You can use date_of_creation and date_of_cessation.
- now grab only the company_numbers of companies that are still active, and those that existed for over 10 years
- use the personsWithSignificantControl API (mentioned above) to find all significant people in these companies. What can you say about their nationalities?

-  ... see what else you can find in the data

In [ ]:
# api request code to get you started

# !pip install requests  # you might have to uncomment and run this if you're on anaconda
import requests


# generic function that will call API for you, given a url ending
def call_api_with(url_extension):
    your_company_house_api_key =".......here paste the api key from steps above......."
    
    login_headers = {"Authorization":your_company_house_api_key}
    url = f"https://api.companieshouse.gov.uk/{url_extension}"
    print(f'requesting: {url}')
    res = requests.get(url, headers=login_headers)
    return res.json()


# some functions to get you started:

def get_one_test_company_or_error():
    url = f"search/companies?q=shop&items_per_page=1"
    return call_api_with(url)

def search_for_companies_with_query(query):
    url = f"search/companies?q={query}&items_per_page=100"
    return call_api_with(url).get('items', [])

def data_for_company(company_number):
    url = f"company/{company_number}"
    return call_api_with(url)

def all_persons_in_company(company_number):
    url = f"company/{company_number}/persons-with-significant-control"
    return call_api_with(url).get('items', [])

In [ ]:
get_one_test_company_or_error()

In [ ]:
# below you can start working on the tasks:

# - find all information of companies with a word 'Bagel' in the name (or just use any other word)

bagel_companies = search_for_companies_with_query("bagels")
bagel_companies

### - from all those companies, how would you get those that are still active? can you calculate how old they are on average?



### - and for those that are not active, how long did they exist? You can use date_of_creation and date_of_cessation.

### - now grab only the company_numbers of companies that are still active, and those that existed for over 10 years

### - use the personsWithSignificantControl API (mentioned above) to find all significant people in these companies. What can you say about their nationalities?

# some example functions for cleaning up the data

In [ ]:
def gender_if_known(person):
    name_elements = person.get('name_elements', {}) # we can use my_dictionary.get(key, value_if_key_missing)
    title_if_known = name_elements.get('title', 'Unknown') 
    
    if title_if_known == 'Mr':
        return 'Male'
    elif title_if_known == 'Mrs' or  title_if_known == 'Ms'  or  title_if_known == 'Miss': # there can be more
        return 'Female'
    else:
        return 'Other or Unknown'
    

assert gender_if_known( {'name': 'Mrs Jasvinder Sohal', 'name_elements': { 'title': 'Mrs'}})  == 'Female'
assert gender_if_known( {'name': 'Mr Jasvinder Sohal', 'name_elements': { 'title': 'Mr'}})  == 'Male'
assert gender_if_known( {'name': 'Dr Jasvinder Sohal', 'name_elements': { 'title': 'Dr'}})  == 'Other or Unknown'

In [ ]:
def int_from_voting_string(voting_string):
    if voting_string.count("voting"):
        return int(voting_string.split("-")[4])
    else:
        return 0
#     input: 'voting-rights-25-to-50-percent'
#     output: 50

assert int_from_voting_string('voting-rights-25-to-50-percent') == 50
assert int_from_voting_string('voting-rights-0-to-25-percent') == 25
assert int_from_voting_string('ownership-of-shares-25-to-50-percent') == 0

In [ ]:
def number_of_voting_rights(all_rights):
    voting_percents = [int_from_voting_string(one_right)
                       for one_right in all_rights
                       if one_right.count("voting")]
    return sum(voting_percents)
    #     eg. input: ['ownership-of-shares-75-to-100-percent',
    #    'voting-rights-25-to-50-percent',
    #    'right-to-appoint-and-remove-directors']
    #     
    #     output: 50
    
assert number_of_voting_rights(['ownership-of-shares-75-to-100-percent','voting-rights-25-to-50-percent']) == 50
assert number_of_voting_rights(['ownership-of-shares-75-to-100-percent','voting-rights-0-to-25-percent']) == 25
assert number_of_voting_rights(['ownership-of-shares-75-to-100-percent']) == 0